In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import interp2d

## Definitions

+ I call `boat_velocity` to the equivalent velocity of the boat if there were not streams. `boat_velocity` is a tuple for both the angle (`theta`, in radians) and the `velocity_mod` (in the same units as the `stream_velocities`)
+ I call `stream_velocity` to the velocity of the stream. 
+ I call `velocity` to the composition between both velocities

In [104]:
def interpolated_value(lat, long, vel_array, xy_positions, step = 0.01):
    """Returns the interpolated value of the tensor for the latitude and longitude requested.
    
    """
    
    idx = (np.abs(xy_positions[0] - lat)).argmin()
    idy = (np.abs(xy_positions[1] - long)).argmin()
    
    
    if xy_positions[0][idx] < lat: idx += 1
    if xy_positions[1][idy] < long: idy += 1
        
    x = xy_positions[0][idx:idx+2]
    y = xy_positions[1][idy:idy+2]
    
    small_array = vel_array[idx:idx+2,idy:idy+2,:]

    velx = interp2d(x, y, small_array[:,:,0])
    
    vely = interp2d(x, y, small_array[:,:,1])
    
    new_velx = velx(lat, long)
    new_vely = vely(lat, long)
    
    #Interpolated square
    lat_s = np.arange(xy_positions[0][idx], xy_positions[0][idx+1], step)
    long_s = np.arange(xy_positions[1][idx], xy_positions[1][idx+1], step)
    velx_s = velx(lat_s, long_s)
    vely_s = vely(lat_s, long_s)
    
    
    
    return new_velx, new_vely, lat_s, long_s, velx_s, vely_s

In [105]:
def velocity_composition(boat_velocity, boat_position, stream_velocitites, xy_stream_velocities):
    
    """Given the boat velocity and the stream velocity, computes the total velocity of the boat.
    """
    lat_boat, long_boat = boat_position
    velx, vely, *_ = interpolated_value(lat_boat, long_boat, stream_velocitites, xy_stream_velocities)
    
    return boat_velocity[0] + velx, boat_velocity[1] + vely

In [109]:
def boat_movement(boat_velocity, boat_position, stream_velocitites, xy_stream_velocities, ts):
    vx, vy, *_ = velocity_composition(boat_velocity, boat_position, stream_velocitites, xy_stream_velocities)
       
    
    return boat_position[0]+vx*ts, boat_position[1]+vy*ts
    

In [110]:
vel_array = np.random.RandomState(seed = 2).rand(10,20,2)
xy_positions = np.array([np.arange(-5,5), np.arange(-13,7)])
ts = 0.1

/home/borja/.local/share/virtualenvs/ocean-route-optimization-DaCDijO3/lib/python3.7/site-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [112]:
velocity_composition((np.array(11), np.array(3)), (1, 4.2), vel_array, xy_positions)

(array([11.32375058]), array([3.98886209]))

In [113]:
boat_movement((np.array(11), np.array(3)), (1, 4.2), vel_array, xy_positions, ts)

(array([2.13237506]), array([4.59888621]))